In [2]:
import pandas as pd
import os
from Bio import SeqIO

In [3]:
# 设置工作目录
workdir = "."
output_dir = os.path.join(workdir, "7_featurecounts")
genome_file = "hg38.fa"

# 读取hg38基因组序列文件
hg38_sequences = SeqIO.to_dict(SeqIO.parse(genome_file, "fasta"))

# 获取所有 featureCounts 和 CPM 文件
featurecounts_files = [f for f in os.listdir(output_dir) if f.endswith("_featurecounts.txt")]
cpm_files = [item.replace('_featurecounts.txt', '_cpm.txt') for item in featurecounts_files]

In [4]:
featurecounts_files

['BE2-shNC-2-IP_fwd_peaks_peaks_featurecounts.txt',
 'BE2-shMETTL14-2-IP_rev_peaks_peaks_featurecounts.txt',
 'BE2-shNC-1-IP_rev_peaks_peaks_featurecounts.txt',
 'BE2-shMETTL14-1-IP_fwd_peaks_peaks_featurecounts.txt',
 'BE2-shNC-1-IP_fwd_peaks_peaks_featurecounts.txt',
 'BE2-shMETTL14-2-IP_fwd_peaks_peaks_featurecounts.txt',
 'BE2-shNC-2-IP_rev_peaks_peaks_featurecounts.txt',
 'BE2-shMETTL14-1-IP_rev_peaks_peaks_featurecounts.txt']

In [5]:
cpm_files

['BE2-shNC-2-IP_fwd_peaks_peaks_cpm.txt',
 'BE2-shMETTL14-2-IP_rev_peaks_peaks_cpm.txt',
 'BE2-shNC-1-IP_rev_peaks_peaks_cpm.txt',
 'BE2-shMETTL14-1-IP_fwd_peaks_peaks_cpm.txt',
 'BE2-shNC-1-IP_fwd_peaks_peaks_cpm.txt',
 'BE2-shMETTL14-2-IP_fwd_peaks_peaks_cpm.txt',
 'BE2-shNC-2-IP_rev_peaks_peaks_cpm.txt',
 'BE2-shMETTL14-1-IP_rev_peaks_peaks_cpm.txt']

In [6]:
# 定义 DRACH 列表
DRACH_list = ["AAACA", "AAACC", "AAACT", "AGACA", "AGACC", "AGACT", 
              "GAACA", "GAACC", "GAACT", "GGACA", "GGACC", "GGACT", 
              "TAACA", "TAACC", "TAACT", "TGACA", "TGACC", "TGACT"]

In [7]:
# 定义函数以获取碱基序列
def get_sequence(chrom, start, end, strand):
    if chrom not in hg38_sequences:
        print(f"Warning: Chromosome {chrom} not found in genome sequences.")
        return ""
    
    seq = hg38_sequences[chrom].seq[start-1:end]
    if strand == '-':
        seq = seq.reverse_complement()
    return str(seq)

In [8]:
# 修改 replace_drach 函数以返回 DRACH 模式的索引位置
def replace_drach_and_get_index(sequence, drach_list):
    index_list = []
    for drach in drach_list:
        index = sequence.find(drach)
        while index != -1:
            index_list.append(index)
            sequence = sequence[:index+2] + '6' + sequence[index+3:]
            index = sequence.find(drach, index + 1)
    return sequence, index_list

In [9]:
# 确保 featureCounts 文件和 CPM 文件的数量一致
assert len(featurecounts_files) == len(cpm_files), "Mismatch between featureCounts and CPM files count."

In [10]:
# 处理每个配对的 featureCounts 和 CPM 文件
for fc_file, cpm_file in zip(featurecounts_files, cpm_files):
    # 读取 featureCounts 文件
    fc_filepath = os.path.join(output_dir, fc_file)
    fc_df = pd.read_csv(fc_filepath, sep='\t', comment='#')
    
    # 读取 CPM 文件
    cpm_filepath = os.path.join(output_dir, cpm_file)
    cpm_df = pd.read_csv(cpm_filepath, sep='\t')
    
    # 选择 featureCounts 文件中的 Geneid, Chr, Start, End, Strand, Length 列
    fc_selected = fc_df[['Geneid', 'Chr', 'Start', 'End', 'Strand', 'Length']]
    fc_selected['Chr'] = fc_selected['Chr'].apply(lambda x: 'chr' + str(x))
    
    # 设置 Strand 值
    if "fwd" in fc_file:
        fc_selected['Strand'] = "+"
    elif "rev" in fc_file:
        fc_selected['Strand'] = "-"
    
    # 合并 CPM 文件中的数据
    merged_df = pd.merge(fc_selected, cpm_df, on='Geneid')
    
    # 获取 BAM 文件列名
    ip_bam_col = merged_df.columns[-2]
    input_bam_col = merged_df.columns[-1]
    
    # 计算 IP/input 列
    merged_df['IP/input'] = merged_df[ip_bam_col] / merged_df[input_bam_col]
    
    
    # 添加 kmers 列
    merged_df['kmers'] = merged_df.apply(
        lambda row: get_sequence(row['Chr'], row['Start'], row['End'], row['Strand']),
        axis=1
    )
    merged_df['kmers'] = merged_df['kmers'].apply(lambda x: x.upper())
    
    merged_df['kmers_len'] = merged_df['kmers'].apply(len)
    
    # 替换 kmers 列中的 DRACH 模式并获取索引位置
    replaced_info = merged_df['kmers'].apply(lambda x: replace_drach_and_get_index(x, DRACH_list))
    merged_df['kmers'] = replaced_info.apply(lambda x: x[0])
    merged_df['Site'] = replaced_info.apply(lambda x: x[1])
    
    # 根据 Strand 调整 Site 值
    merged_df['Site'] = merged_df.apply(
        lambda row: [row['Start'] + idx for idx in row['Site']], axis=1
    )
    
    # 生成输出文件名
    output_file = os.path.join(output_dir, fc_file.replace("_featurecounts.txt", "_merged.csv"))
    
    # 保存合并后的数据
    merged_df.to_csv(output_file, index=False)
    
    print(f"处理完成，结果保存在: {output_file}")

print("所有文件处理完成。")


/tmp/ipykernel_21700/1352171814.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_selected['Chr'] = fc_selected['Chr'].apply(lambda x: 'chr' + str(x))
/tmp/ipykernel_21700/1352171814.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_selected['Strand'] = "+"


处理完成，结果保存在: ./7_featurecounts/BE2-shNC-2-IP_fwd_peaks_peaks_merged.csv


/tmp/ipykernel_21700/1352171814.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_selected['Chr'] = fc_selected['Chr'].apply(lambda x: 'chr' + str(x))
/tmp/ipykernel_21700/1352171814.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_selected['Strand'] = "-"


处理完成，结果保存在: ./7_featurecounts/BE2-shMETTL14-2-IP_rev_peaks_peaks_merged.csv


/tmp/ipykernel_21700/1352171814.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_selected['Chr'] = fc_selected['Chr'].apply(lambda x: 'chr' + str(x))
/tmp/ipykernel_21700/1352171814.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_selected['Strand'] = "-"


...

处理完成，结果保存在: ./7_featurecounts/BE2-shMETTL14-2-IP_fwd_peaks_peaks_merged.csv


/tmp/ipykernel_21700/1352171814.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_selected['Chr'] = fc_selected['Chr'].apply(lambda x: 'chr' + str(x))
/tmp/ipykernel_21700/1352171814.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_selected['Strand'] = "-"


处理完成，结果保存在: ./7_featurecounts/BE2-shNC-2-IP_rev_peaks_peaks_merged.csv


/tmp/ipykernel_21700/1352171814.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_selected['Chr'] = fc_selected['Chr'].apply(lambda x: 'chr' + str(x))
/tmp/ipykernel_21700/1352171814.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_selected['Strand'] = "-"


处理完成，结果保存在: ./7_featurecounts/BE2-shMETTL14-1-IP_rev_peaks_peaks_merged.csv
所有文件处理完成。


In [10]:
merged_df

,Geneid,Chr,Start,End,Strand,Length,./4_hisat2/MAPQ20/split/BE2-shMETTL14-1-IP.rev.bam,./4_hisat2/MAPQ20/split/BE2-shMETTL14-1-input.rev.bam,IP/input,kmers,kmers_len,Site
0,0,chr1,629844,630056,-,213,132.131459,75.904204,1.740766,GATGGGGGCTAGTTTTTGTCATGTGAGAAGGAGCAGGCCGGATGTC...,213,"[629915, 629906]"
1,1,chr1,633999,634199,-,201,5719.325734,4816.054316,1.187554,GTGGCCTGCAGTAATGTTAGCGGTTAGGCGTACGGCCAGGGCTATT...,201,[634076]
2,2,chr1,912529,912762,-,234,12.488665,4.169213,2.995449,TCTGCCAAAGGCTCCCGAGCCTCATAGTTCAGGAGGGGCAGGAGGG...,234,"[912621, 912594, 912741]"
3,3,chr1,913004,913207,-,204,5.933841,2.268542,2.615706,GCTCCCTCCCTGCCAGGCACAGTTCACAGAGGCAGCTGCTTTGGGA...,204,[913061]
4,4,chr1,915268,915469,-,202,13.040651,7.418747,1.757797,GCCCTCTGGCTTCGGGAGTGCTGGGCGCAGTGGTGGCAGGTGCCTC...,202,"[915323, 915438]"
...,...,...,...,...,...,...,...,...,...,...,...,...
4453,4453,chrY,1594188,1594407,-,220,0.000000,0.000000,NaN,GACCCCTACTCCACCTCACGCCTGTGCCTCTGAAATTGG6CAGGCA...,220,"[1594225, 1594236, 1594363, 1594305]"
4454,4454,chrY,1595397,1595730,-,334,61.615349,0.613120,100.494831,GCGCACATAGGG6CATACAAAGTACACACACCGGTTCACAGGCACA...,334,"[1595518, 1595407, 1595557]"
4455,4455,chrY,19567498,19567698,-,201,4.967867,3.678717,1.350434,CTAGCCTGTGTCCCAAGTAAGCCCTAGACGGTGGTG6CCAGCACCG...,201,"[19567562, 19567532]"
4456,4456,chrY,19587272,19587513,-,242,23.183379,10.852217,2.136281,TATTACCTAAG6CTTTTTCTTTGTTCCTGTTGCATTGTAAAGCTTG...,242,"[19587281, 19587319, 19587363]"
